In [3]:
import numpy as np
import h5py
import time
import pandas as pd
import math
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
h5_path = '/home/jan/Documents/Trainingsdaten/ZPs/ZP1/ZP_1_full_part.h5'
part_name = 'ZP1_combined'
slice_name = "Slice00001"
num_slice = 1590

In [9]:
def get_2D_data_from_h5_filtered_np_xy_changed(h5_path, part_name, slice_name, show_info = False):
    # opening h5 and getting the data
    start_time = time.time()

    with h5py.File(h5_path, 'r') as h5:
        # check whether slice exists -> if not: empty array returned
        # here the X and Y axis are changed to fit the OpenCV coordinate system
        
        if slice_name in h5[part_name]:
            X_Axis = np.array(h5[part_name][slice_name]['Y-Axis'][:]).astype(int)
            Area = np.array(h5[part_name][slice_name]['Area'][:]).astype(int)
            Intensity = np.array(h5[part_name][slice_name]['Intensity'][:]).astype(int)
            Y_Axis = np.array(h5[part_name][slice_name]['X-Axis'][:]).astype(int)

            X_Axis_size = X_Axis.size
            Y_Axis_size = Y_Axis.size
            Area_size = Area.size
            Intensity_size = Intensity.size

            # if dimensions aren't equal the following code block is entered
            if not X_Axis_size == Y_Axis_size == Area_size == Intensity_size:

                # determine the lowest value among the different sizes
                size_arr = np.array([X_Axis_size, Y_Axis_size, Area_size, Intensity_size])
                min_size = size_arr.min()

                if X_Axis_size != min_size:
                    diff_size_x = X_Axis_size - min_size  # calculating the difference between the actual value and the minimum and substracting it from the array
                    X_Axis_new = np.delete(X_Axis, -diff_size_x)
                    X_Axis = X_Axis_new
                    X_Axis_size = X_Axis.size

                if Y_Axis_size != min_size:
                    diff_size_y = Y_Axis_size - min_size
                    Y_Axis_new = np.delete(Y_Axis, -diff_size_y)
                    Y_Axis = Y_Axis_new
                    Y_Axis_size = Y_Axis.size

                if Area_size != min_size:
                    diff_size_area = Area_size - min_size
                    Area_new = np.delete(Area, -diff_size_area)
                    Area = Area_new
                    Area_size = Area.size

                if Intensity_size != min_size:
                    diff_size_intensity = Intensity_size - min_size
                    Intensity_new = np.delete(Intensity, -diff_size_intensity)
                    Intensity = Intensity_new
                    Intensity_size = Intensity.size

                # by reducing all the dimensions to the minimum equal dimensions are guaranteed
                # there is a risk of deleting more than just one datapoint without noticing -> maybe add an alert after more than 5(?) while iterations
            #

            if show_info:
                print(str(X_Axis_size) + ' total data points found')

            combos = np.stack((X_Axis, Y_Axis, Area, Intensity), axis=-1)

            # filtering out the outlier data points
            median_x = np.median(combos[:, 0])
            median_y = np.median(combos[:, 1])
            std_x = int(combos[:, 0].std())
            std_y = int(combos[:, 1].std())
            low_limit_x = median_x - 2 * std_x
            low_limit_y = median_y - 2 * std_y
            high_limit_x = median_x + 2 * std_x
            high_limit_y = median_y + 2 * std_y

            combos = np.delete(combos, np.where(combos[:, 0] < low_limit_x), axis=0)
            combos = np.delete(combos, np.where(combos[:, 0] > high_limit_x), axis=0)
            combos = np.delete(combos, np.where(combos[:, 1] < low_limit_y), axis=0)
            combos = np.delete(combos, np.where(combos[:, 1] > high_limit_y), axis=0)

            # filtering out the data points where area and intensity are =0
            area_zeros = np.where(combos[:, 2] == 0)
            intensity_zeros = np.where(combos[:, 3] == 0)
            zero_area_intensity_indices = np.intersect1d(area_zeros,
                                                         intensity_zeros)  # array of indices where area AND intensity are = 0

            # deleting all the datapoints where area AND intensity are = 0
            combos_wo_only_zeros = np.delete(combos, zero_area_intensity_indices, axis=0)
            if show_info:
                print(str(combos_wo_only_zeros.shape[0]) + ' data points where area != 0 AND intensity != 0')

            combos_wo_only_zeros_unique, unique_indices = np.unique(combos_wo_only_zeros[:, [0, 1]], axis=0,
                                                                    return_index=True)
            combos_unique = combos_wo_only_zeros[unique_indices]

            if show_info:
                print(str(combos_unique.shape[0]) + ' unique data points where area != 0 AND intensity != 0')

            Index_range = np.arange(combos_wo_only_zeros.shape[0])
            indices_of_interest = np.setdiff1d(Index_range,
                                               unique_indices)  # all the indices belonging to non unique x,y-combinations

            combo_processed_array = np.empty([0, 4], dtype=int)
            start_time = time.time()
            combos_wo_only_zeros_copy = np.copy(combos_wo_only_zeros)
            index_counter = 0
            shape_counter = 0
            indices_list = []

            if show_info:
                print("vor iterieren %s seconds ---" % (time.time() - start_time))

            for index in indices_of_interest:
                xy_combo = combos_wo_only_zeros[:, [0, 1]][index]
                if \
                np.where((combo_processed_array[:, 0] == xy_combo[0]) * (combo_processed_array[:, 1] == xy_combo[1]))[
                    0].size == 0:
                    index_counter += 1
                    xy_combo = combos_wo_only_zeros[:, [0, 1]][index]
                    indices_relevant = \
                    np.where((combos_wo_only_zeros[:, 0] == xy_combo[0]) * (combos_wo_only_zeros[:, 1] == xy_combo[1]))[
                        0]
                    max_area_of_combo = np.amax(combos_wo_only_zeros[:, 2][indices_relevant])
                    max_intensity_of_combo = np.amax(combos_wo_only_zeros[:, 3][indices_relevant])

                    max_combos = np.stack((xy_combo[0], xy_combo[1], max_area_of_combo, max_intensity_of_combo),
                                          axis=-1)

                    combos_wo_only_zeros_copy = np.vstack((combos_wo_only_zeros_copy, max_combos))
                    shape_counter += indices_relevant.shape[0]
                    indices_list.append(list(indices_relevant))

                    combo_processed_array = np.vstack((combo_processed_array, max_combos))

            indices_relevant = np.hstack(indices_list)
            combos_wo_only_zeros_copy = np.delete(combos_wo_only_zeros_copy, indices_relevant, axis=0)
        else:
            combos_wo_only_zeros_copy = np.empty([0, 4], dtype=int)
            print('{} is not existing -> empty array created'.format(slice_name))

        if show_info:
            print("array creation took %s seconds ---" % (time.time() - start_time))
        return combos_wo_only_zeros_copy

In [22]:
# get the maximum intensity for whole part
def get_max_intensity_whole_part(h5_path, part_name,  max_slice_num):
    df = pd.DataFrame(columns=['Slice_num', 'maxInt',  'medianInt', 'meanInt', 'StdInt'])

    for num_slice in range(max_slice_num):
        slice_name = 'Slice' + str("{:05d}".format(num_slice + 1))

        array = get_2D_data_from_h5_filtered_np_xy_changed(h5_path, part_name, slice_name)
        maxInt = array[:, 3].max()
        medianInt = np.median(array[:, 3])
        meanInt = np.mean(array[:, 3])
        stdInt = np.std(array[:, 3])
        
        df = df.append({'Slice_num': "{:05d}".format(num_slice + 1), 'maxInt': maxInt, 'medianInt':medianInt, 'meanInt':meanInt, 'StdInt':stdInt}, ignore_index=True)

    return df, df['medianInt'].mean(), df['StdInt'].mean()

In [23]:
df, mean_medianInt, mean_std = get_max_intensity_whole_part(h5_path, part_name,  1593)

In [24]:
df

,Slice_num,maxInt,medianInt,meanInt,StdInt
0,00001,5623,1095.0,1231.146987,384.988228
1,00002,1851,897.0,909.575857,97.682933
2,00003,4518,897.0,948.993002,199.443379
3,00004,2154,840.0,851.102184,80.301922
4,00005,2656,852.0,871.920199,107.755099
...,...,...,...,...,...
1588,01589,2220,841.0,862.603020,110.773470
1589,01590,3721,831.0,856.373980,131.204939
1590,01591,3290,838.0,858.654228,115.140395
1591,01592,1217,793.5,757.433437,125.294295


In [20]:
df['medianInt'].mean()

845.1541117388575

In [21]:
mean_medianInt

845.1541117388575

In [8]:
df = get_max_intensity_whole_part(h5_path, part_name,  1593)

In [9]:
df_relevant_slices = df.iloc[183:1592]

In [10]:
df_relevant_slices

,Slice_num,maxInt,maxArea,medianInt,meanInt,StdInt
183,00184,1694,948,842.0,847.864136,67.965304
184,00185,2431,1157,855.0,866.645325,83.271709
185,00186,1830,940,844.0,854.411858,74.487917
186,00187,1908,1275,864.0,882.008144,97.858057
187,00188,2073,1082,840.0,848.146151,71.235500
...,...,...,...,...,...,...
1587,01588,1808,1077,825.0,832.792987,70.970657
1588,01589,2220,1210,841.0,862.603020,110.773470
1589,01590,3721,1717,831.0,856.373980,131.204939
1590,01591,3290,1438,838.0,858.654228,115.140395
